<a href="https://colab.research.google.com/github/abhi271998/Transfer-learning-usingRestNet50-for-classification-/blob/main/Transfer_learning_restnet50.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import numpy as np
import pandas as pd
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Dense, Activation, GlobalAveragePooling2D, Dropout, Input
from tensorflow.keras.preprocessing import image
from tensorflow.keras.utils import to_categorical
from google.colab import drive

In [ ]:
drive.mount('/content/MyDrive/',force_remount=True)

Mounted at /content/MyDrive/


In [ ]:
!unzip '/content/MyDrive/MyDrive/Transfer_Learning/data.zip'

Archive:  /content/MyDrive/MyDrive/Transfer_Learning/data.zip
   creating: data/
   creating: data/cats/
  inflating: data/cats/cat.1.jpg     
  inflating: data/cats/cat.10.jpg    
  inflating: data/cats/cat.100.jpg   
  inflating: data/cats/cat.101.jpg   
  inflating: data/cats/cat.102.jpg   
  inflating: data/cats/cat.103.jpg   
  inflating: data/cats/cat.104.jpg   
  inflating: data/cats/cat.105.jpg   
  inflating: data/cats/cat.106.jpg   
  inflating: data/cats/cat.107.jpg   
  inflating: data/cats/cat.108.jpg   
  inflating: data/cats/cat.109.jpg   
  inflating: data/cats/cat.11.jpg    
  inflating: data/cats/cat.110.jpg   
  inflating: data/cats/cat.111.jpg   
  inflating: data/cats/cat.112.jpg   
  inflating: data/cats/cat.113.jpg   
  inflating: data/cats/cat.114.jpg   
  inflating: data/cats/cat.115.jpg   
  inflating: data/cats/cat.116.jpg   
  inflating: data/cats/cat.117.jpg   
  inflating: data/cats/cat.118.jpg   
  inflating: data/cats/cat.119.jpg   
  inflating: data/cat

In [ ]:
folder_path = '/content/data/'

In [ ]:
import os

In [ ]:
dirs = os.listdir(folder_path)
print(dirs)

['Humans', 'cats', 'horses', 'dogs']


In [ ]:
label_dict = {'Humans' : 0, 'dogs' : 1, 'cats' : 2, 'horses' : 3}

In [ ]:
image_data = []
labels = []

for ix in dirs:
  path = folder_path + ix + '/'
  for img in os.listdir(path):
    imag = image.load_img(path+img, target_size=(224, 224))
    imag_arr = image.img_to_array(imag)
    image_data.append(imag_arr)
    labels.append(label_dict[ix])

image_data = np.array(image_data)
labels = np.array(labels)

print(image_data.shape, labels.shape)

(808, 224, 224, 3) (808,)


In [ ]:
combined = list(zip(image_data, labels))
import random
random.shuffle(combined)

image_data[:], labels[:] = zip(*combined)

In [ ]:
Y = to_categorical(labels)
print(Y.shape)

(808, 4)


In [ ]:
from tensorflow.keras.applications.resnet50 import ResNet50

In [ ]:
model_up = ResNet50(include_top=False, weights='imagenet', input_shape=(224, 224, 3))

model_up.summary()

94773248/94765736 [==============================] - 1s 0us/step
Model: "resnet50"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, 224, 224, 3) 0                                            
__________________________________________________________________________________________________
conv1_pad (ZeroPadding2D)       (None, 230, 230, 3)  0           input_1[0][0]                    
__________________________________________________________________________________________________
conv1_conv (Conv2D)             (None, 112, 112, 64) 9472        conv1_pad[0][0]                  
__________________________________________________________________________________________________
conv1_bn (BatchNormalization)   (None, 112, 112, 64) 256         conv1_conv[0][0]                 
__________________________

In [ ]:
av1 = GlobalAveragePooling2D()(model_up.output)
fc1 = Dense(256, activation='relu')(av1)
dp1 = Dropout(0.25)(fc1)
fc2 = Dense(4, activation='softmax')(dp1)

model_new = Model(inputs = model_up.input, outputs = fc2)

model_new.summary()

Model: "model"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, 224, 224, 3) 0                                            
__________________________________________________________________________________________________
conv1_pad (ZeroPadding2D)       (None, 230, 230, 3)  0           input_1[0][0]                    
__________________________________________________________________________________________________
conv1_conv (Conv2D)             (None, 112, 112, 64) 9472        conv1_pad[0][0]                  
__________________________________________________________________________________________________
conv1_bn (BatchNormalization)   (None, 112, 112, 64) 256         conv1_conv[0][0]                 
______________________________________________________________________________________________

In [ ]:
from tensorflow.keras.optimizers import Adam
model_new.compile(loss = 'categorical_crossentropy', optimizer=Adam(0.00003), metrics=['accuracy'])


In [ ]:
for ix in range(len(model_new.layers)):
  print(ix, model_new.layers[ix])

0 <tensorflow.python.keras.engine.input_layer.InputLayer object at 0x7f527378be50>
1 <tensorflow.python.keras.layers.convolutional.ZeroPadding2D object at 0x7f52730d7950>
2 <tensorflow.python.keras.layers.convolutional.Conv2D object at 0x7f5273789a10>
3 <tensorflow.python.keras.layers.normalization_v2.BatchNormalization object at 0x7f5273721e90>
4 <tensorflow.python.keras.layers.core.Activation object at 0x7f5272fd4dd0>
5 <tensorflow.python.keras.layers.convolutional.ZeroPadding2D object at 0x7f52605d7610>
6 <tensorflow.python.keras.layers.pooling.MaxPooling2D object at 0x7f526055af90>
7 <tensorflow.python.keras.layers.convolutional.Conv2D object at 0x7f526056f490>
8 <tensorflow.python.keras.layers.normalization_v2.BatchNormalization object at 0x7f52605631d0>
9 <tensorflow.python.keras.layers.core.Activation object at 0x7f5260581d90>
10 <tensorflow.python.keras.layers.convolutional.Conv2D object at 0x7f5260586dd0>
11 <tensorflow.python.keras.layers.normalization_v2.BatchNormalization o

In [ ]:
for ix in range(175):
  model_new.layers[ix].trainable = False

model_new.summary()

Model: "model"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, 224, 224, 3) 0                                            
__________________________________________________________________________________________________
conv1_pad (ZeroPadding2D)       (None, 230, 230, 3)  0           input_1[0][0]                    
__________________________________________________________________________________________________
conv1_conv (Conv2D)             (None, 112, 112, 64) 9472        conv1_pad[0][0]                  
__________________________________________________________________________________________________
conv1_bn (BatchNormalization)   (None, 112, 112, 64) 256         conv1_conv[0][0]                 
______________________________________________________________________________________________

In [ ]:
hist = model_new.fit(image_data, Y, batch_size=16, epochs=5, validation_split=0.2)

Epoch 1/5
41/41 [==============================] - 47s 230ms/step - loss: 1.5968 - accuracy: 0.3632 - val_loss: 2.1577 - val_accuracy: 0.2716
Epoch 2/5
41/41 [==============================] - 7s 175ms/step - loss: 0.8997 - accuracy: 0.6261 - val_loss: 2.4142 - val_accuracy: 0.2531
Epoch 3/5
41/41 [==============================] - 7s 175ms/step - loss: 0.7420 - accuracy: 0.6810 - val_loss: 2.6230 - val_accuracy: 0.2469
Epoch 4/5
41/41 [==============================] - 7s 176ms/step - loss: 0.7116 - accuracy: 0.6525 - val_loss: 2.9052 - val_accuracy: 0.2778
Epoch 5/5
41/41 [==============================] - 7s 177ms/step - loss: 0.6391 - accuracy: 0.6791 - val_loss: 3.1807 - val_accuracy: 0.2716
